In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import GRU_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [ ]:
conf = Config()
load_models = GRU_Models()
teacher_model = load_models.gru_4
teacher_model.load()
student_model = load_models.light_gru_4
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="feature",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/gru_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513, 1]) torch.Size([512])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 68.79%, Train loss: 0.0091, Val loss: 0.9555
Epoch: 2/10, Accuracy: 74.70%, Train loss: 0.0058, Val loss: 0.5497
Epoch: 3/10, Accuracy: 81.05%, Train loss: 0.0042, Val loss: 0.3811
Epoch: 4/10, Accuracy: 83.97%, Train loss: 0.0034, Val loss: 0.2914
Epoch: 5/10, Accuracy: 86.25%, Train loss: 0.0029, Val loss: 0.2468
Epoch: 6/10, Accuracy: 87.19%, Train loss: 0.0027, Val loss: 0.2233
Epoch: 7/10, Accuracy: 88.01%, Train loss: 0.0026, Val loss: 0.2056
Epoch: 8/10, Accuracy: 88.51%, Train loss: 0.0025, Val loss: 0.1968
Epoch: 9/10, Accuracy: 88.68%, Train loss: 0.0024, Val loss: 0.1914
Epoch: 10/10, Accuracy: 89.25%, Train loss: 0.0023, Val loss: 0.1825


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.65%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 89.12%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_gru_1.pth
